Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

=====> /Users/rob/.julia/dev/StatisticalRethinking/scripts/04



Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)


make: `/Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights' is up to date.
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD        Naive SE       MCSE      ESS
alpha 154.59573750 0.26995281 0.00426832878 0.0045302188 1000
 beta   0.90612336 0.04237026 0.00066993263 0.0006183675 1000
sigma   5.09915770 0.19127638 0.00302434507 0.0032764362 1000

Quantiles:
          2.5%        25.0%       50.0%      75.0%       97.5%   
alpha 154.06495000 154.4110000 154.599000 154.7742500 155.1260250
 beta   0.82422848   0.8767107   0.905524   0.9348765   0.9885438
sigma   4.73503650   4.9697400   5.097975   5.2245600   5.4957670



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.368 0.89325 4.75385; 154.331 0.875229 4.69409; … ; 154.437 0.929291 5.18605; 154.795 0.904508 4.95427]

Union{Missing, Float64}[154.682 0.88841 4.64073; 154.666 0.904883 4.72027; … ; 154.429 0.874475 4.94093; 154.286 0.870871 5.45014]

Union{Missing, Float64}[154.942 0.943534 5.17146; 154.352 0.801197 5.10677; … ; 154.774 0.956178 5.09527; 154.639 0.973743 5.12215]

Union{Missing, Float64}[154.601 0.962101 5.25464; 154.739 0.808237 5.03038; … ; 154.13 0.971662 5.38316; 154.663 0.931617 5.17603]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD        Naive SE       MCSE      ESS
alpha 154.59573750 0.26995281 0.00426832878 0.0045302188 1000
 beta   0.90612336 0.04237026 0.00066993263 0.0006183675 1000
sigma   5.09915770 0.19127638 0.00302434507 0.0032764362 1000

Quantiles:
          2.5%        25.0%       50.0%      75.0%       97.5%   
alpha 154.06495000 154.4110000 154.599000 154.7742500 155.1260250
 beta   0.82422848   0.8767107   0.905524   0.9348765   0.9885438
sigma   4.73503650   4.9697400   5.097975   5.2245600   5.4957670



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*